# MOVIES PROJECT

necesary imports

In [42]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import re

get responses from lots of urls <br>
starting with: <br>
https://www.imdb.com/...&start=1 <br>
up to: <br>
https://www.imdb.com/...&start=6651

In [43]:
responses = []
# for position in range(1, 6652, 50):
for position in range(1, 152, 50):
    request = requests.get(
        f"https://www.imdb.com/search/title/?title_type=feature&num_votes=10000,&countries=us&sort=user_rating,desc&start={position}"
    )
    responses.append(request)
len(responses)

4

sopify all of them

In [44]:
soups = []
for response in responses:
    soups.append(BeautifulSoup(response.text, 'html.parser'))
len(soups)


4

## RELEASE DATES

date cleaning function

In [45]:
def extract_int(string):
    return re.search(r"\d+", string).group()

traverse the soups in search for dates, create a list of them, serialise it

In [46]:
movie_release_dates = []
for soup in soups:
    movie_release_dates += soup.findAll('span', class_= 'lister-item-year text-muted unbold')

rel_dates_list = []
for str_date in movie_release_dates:
    rel_dates_list.append( extract_int(str_date.text) )

rel_dates_series = pd.Series(rel_dates_list, name="Release Date")
rel_dates_series

0      1994
1      1972
2      2008
3      1974
4      1957
       ... 
195    1930
196    1928
197    1928
198    1927
199    1926
Name: Release Date, Length: 200, dtype: object

## RUNTIMES

create a time-cleaning function

In [47]:
def clean_time(time_string):

    time_string = time_string.split()
    time_string = time_string[0]

    return int(time_string)

traverse the soups in search for runtimes, create a list of them, serialise it

In [48]:
runtimes = []
for soup in soups:
    runtimes += soup.findAll('span', class_="runtime")

movie_runtimes = []
for runtime in runtimes:
    movie_runtimes.append( clean_time(runtime.text) )
runtimes_series = pd.Series(movie_runtimes, name="Runtime")

runtimes_series

0      142
1      175
2      152
3      202
4       96
      ... 
195    152
196     72
197     76
198     94
199     67
Name: Runtime, Length: 200, dtype: int64

## MOVIE NAMES
and addresses for details pages

traverse the soups in search for movie names and urls, create a list of them, serialise it

In [49]:
movie_headers = []
for soup in soups:
    movie_headers += soup.find_all(name="h3", class_="lister-item-header")

names_list = []
hrefs_list = []     # this is very important for later. they look like this: '/title/tt0111161/'
for header in movie_headers:
    names_list.append(header.contents[3].text)
    hrefs_list.append(header.contents[3]['href'] )

names_series = pd.Series(names_list, name="Movie name")
names_series

0          Skazani na Shawshank
1              Ojciec chrzestny
2                Mroczny Rycerz
3           Ojciec chrzestny II
4      Dwunastu gniewnych ludzi
                 ...           
195      Na Zachodzie bez zmian
196                        Cyrk
197           Człowiek z kamerą
198               Wschód słońca
199                     Generał
Name: Movie name, Length: 200, dtype: object

## FILMING DATES
this may be tricky

first we get the responses for all the 6600+ movies... 

In [51]:
href_responses = []
# hl = hrefs_list[:1]
for href in hrefs_list:
    request = requests.get(
        f"https://www.imdb.com{href}locations"
    )
    href_responses.append(request)
len(href_responses)

200

now let's soupify them...

In [66]:
href_soups = []
for response in href_responses:
    href_soups.append(BeautifulSoup(response.text, 'html.parser'))
len(href_soups)

200

and start digging for dates...

In [124]:
filming_dates = []
for idx, soup in enumerate(href_soups[:2]):
    date = soup.find_all(name="section", id="filming_dates")
    filming_dates += date
len(filming_dates)

<ipython-input-124-c270f2100e50>:7: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  shit = pd.Series()


0

In [112]:
for idx, date in enumerate(filming_dates):
    print(idx, date.contents[3].text.strip())

0 16 June 1993 - 10 September 1993
1 29 March 1971 - 6 August 1971


## MOVIE TYPES

traverse the soups in search for movie names, clean them, create a list of them and serialise it

In [29]:
genres = []

for soup in soups:
    genres += soup.find_all(name="span", class_="genre")

genres_list = []
for gen in genres:
    string = list(gen)[0].replace("\n", "").strip()
    genres_list.append(string)

genres_series = pd.Series(genres_list, name="Genre")
genres_series

0                          Drama
1                   Crime, Drama
2           Action, Crime, Drama
3                   Crime, Drama
4                   Crime, Drama
                 ...            
195                   Drama, War
196              Comedy, Romance
197        Comedy, Drama, Family
198               Drama, Romance
199    Action, Adventure, Comedy
Name: Genre, Length: 200, dtype: object

## RATING

traverse the soup, get stuff from it and serialise it in 2 steps

In [30]:
# <div class="inline-block ratings-imdb-rating" name="ir" data-value="8,3">
ratings = []
for soup in soups:
    ratings += soup.find_all(name="div", attrs={'name':'ir'})

ratings_series = pd.Series([ float(rt.text.strip()) for rt in ratings ], name="Rating")
ratings_series

0      9.3
1      9.2
2      9.0
3      9.0
4      9.0
      ... 
195    8.1
196    8.1
197    8.1
198    8.1
199    8.1
Name: Rating, Length: 200, dtype: float64

## PROFIT!

In [31]:
all_series = [names_series, ratings_series, rel_dates_series, runtimes_series, genres_series]

df = pd.concat(all_series, axis=1)

# %timeit df.loc[df.loc[:,'Genre'].str.contains('Action')]
# %timeit df.loc[df['Genre'].str.contains('Action')]
# %timeit df.loc[ [('Action' in genre) for genre in df.loc[:,'Genre']]]
# %timeit df.loc[ [('Action' in genre) for genre in df['Genre']]]


In [32]:
df

,Movie name,Rating,Release Date,Runtime,Genre
0,Skazani na Shawshank,9.3,1994,142,Drama
1,Ojciec chrzestny,9.2,1972,175,"Crime, Drama"
2,Mroczny Rycerz,9.0,2008,152,"Action, Crime, Drama"
3,Ojciec chrzestny II,9.0,1974,202,"Crime, Drama"
4,Dwunastu gniewnych ludzi,9.0,1957,96,"Crime, Drama"
...,...,...,...,...,...
195,Na Zachodzie bez zmian,8.1,1930,152,"Drama, War"
196,Cyrk,8.1,1928,72,"Comedy, Romance"
197,Człowiek z kamerą,8.1,1928,76,"Comedy, Drama, Family"
198,Wschód słońca,8.1,1927,94,"Drama, Romance"


extract action movies

In [33]:
action = df.loc[ [('Action' in genre) for genre in df['Genre']]]
action

,Movie name,Rating,Release Date,Runtime,Genre
2,Mroczny Rycerz,9.0,2008,152,"Action, Crime, Drama"
5,Władca pierścieni: Powrót króla,8.9,2003,201,"Action, Adventure, Drama"
8,Incepcja,8.8,2010,148,"Action, Adventure, Sci-Fi"
10,Władca pierścieni: Drużyna Pierścienia,8.8,2001,178,"Action, Adventure, Drama"
12,Władca pierścieni: Dwie wieże,8.7,2002,179,"Action, Adventure, Drama"
13,Matrix,8.7,1999,136,"Action, Sci-Fi"
15,Gwiezdne wojny: Część V - Imperium kontratakuje,8.7,1980,124,"Action, Adventure, Fantasy"
22,Gwiezdne wojny,8.6,1977,121,"Action, Adventure, Fantasy"
30,Gladiator,8.5,2000,155,"Action, Adventure, Drama"
33,Léon zawodowiec,8.5,1994,110,"Action, Crime, Drama"
